In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import torch
from external.metrics import compute_all_metrics, jsd_between_point_cloud_sets as JSD, emd_cd
from src.data.point_operation import normalize_point_cloud

def result_dict_tensor_to_np(result_dict):
    for key in result_dict.keys():
        if torch.is_tensor(result_dict[key]):
            result_dict[key] = result_dict[key].item()
    return result_dict

### Reconstruction

In [ ]:
gdt_airplane = torch.tensor(np.load("./out/reconstruction/gdt_airplane.npy")).cuda()
gdt_chair = torch.tensor(np.load("./out/reconstruction/gdt_chair.npy")).cuda()
gdt_car = torch.tensor(np.load("./out/reconstruction/gdt_car.npy")).cuda()

ours_airplane = torch.tensor(np.load("./out/reconstruction/ours_airplane.npy")).cuda()
ours_chair = torch.tensor(np.load("./out/reconstruction/ours_chair.npy")).cuda()
ours_car = torch.tensor(np.load("./out/reconstruction/ours_car.npy")).cuda()

print("gdt_airplane: ", gdt_airplane.shape, "ours_airplane: ", ours_airplane.shape)
print("gdt_chair: ", gdt_chair.shape, "ours_chair: ", ours_chair.shape)
print("gdt_car: ", gdt_car.shape, "ours_car: ", ours_car.shape)

In [ ]:
AE_airplane_results = emd_cd(ours_airplane, gdt_airplane, 128, accelerated_cd=True)
AE_chair_results = emd_cd(ours_chair, gdt_chair, 128, accelerated_cd=True)
AE_car_results = emd_cd(ours_car, gdt_car, 128, accelerated_cd=True)

print("AE_airplane_results:")
print(result_dict_tensor_to_np(AE_airplane_results))
print("AE_chair_results:")
print(result_dict_tensor_to_np(AE_chair_results))
print("AE_car_results:")
print(result_dict_tensor_to_np(AE_car_results))

### Generation

In [ ]:
airplane_gdt = torch.tensor((np.load("./out/samples/generation/gdt_normed_airplane.npy"))).cuda()
chair_gdt = torch.tensor((np.load("./out/samples/generation/gdt_normed_chair.npy"))).cuda()
car_gdt = torch.tensor((np.load("./out/samples/generation/gdt_normed_car.npy"))).cuda()

GEN_airplane_ours = torch.tensor((np.load("./out/samples/generation/ours_airplane.npy"))).cuda()
GEN_car_ours = torch.tensor((np.load("./out/samples/generation/ours_car.npy"))).cuda()
GEN_chair_ours = torch.tensor((np.load("./out/samples/generation/ours_chair.npy"))).cuda()

print("gdt_airplane: ", airplane_gdt.shape, "ours_airplane: ", GEN_airplane_ours.shape)
print("gdt_chair: ", chair_gdt.shape, "ours_chair: ", GEN_chair_ours.shape)
print("gdt_car: ", car_gdt.shape, "ours_car: ", GEN_car_ours.shape)

In [ ]:
GEN_airplane_results = compute_all_metrics(GEN_airplane_ours, airplane_gdt, 256, accelerated_cd=True, compute_nna=True, compute_jsd=True)
GEN_car_results = compute_all_metrics(GEN_car_ours, car_gdt, 256, accelerated_cd=True, compute_nna=True, compute_jsd=True)
GEN_chair_results = compute_all_metrics(GEN_chair_ours, chair_gdt, 256, accelerated_cd=True, compute_nna=True, compute_jsd=True)

print("GEN_airplane_results:")
print(result_dict_tensor_to_np(GEN_airplane_results))
print("GEN_car_results:")
print(result_dict_tensor_to_np(GEN_car_results))
print("GEN_chair_results:")
print(result_dict_tensor_to_np(GEN_chair_results))